In [1]:
!pip install openpyxl==3.1.5
!pip install html5lib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 24.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 18.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
from datetime import datetime
import numpy as np
import os
import re 

In [3]:
# Leemos el html y lo cargamos en una dataframe
tabs = pd.read_html('/work/REPORTE PRINCIPAL.htm')

# Omitimos las primeras 2 columnas
df = tabs[0].iloc[2:]

# Asignamos el primer row como nombre de columnas
df.columns = df.iloc[0]

# Omitimos las siguientes 6 lineas (Hasta donde comienzan las posiciones)
df = df.iloc[6:]
# Omitimos las ultimas 10 lineas
df = df.iloc[:-10].reset_index(drop=True)

# Obtener el index de donde comienzan los Open Trades
open_trades_index = df.apply(lambda row: row.astype(str).str.contains('Open Trades')).idxmax().max()

main_closed_trades_df = df.iloc[:open_trades_index][:-2]
main_open_trades_df = df.iloc[open_trades_index:][2:].reset_index(drop=True)

# Convert Dates Objects into Dates for closed_trades_df, ignoring errors
main_closed_trades_df['Open Time'] = main_closed_trades_df['Open Time'].str.replace('.', '-')
main_closed_trades_df['Open Time'] = pd.to_datetime(main_closed_trades_df['Open Time'], errors='coerce')
main_closed_trades_df['Close Time'] = pd.to_datetime(main_closed_trades_df['Close Time'], errors='coerce')

# Convert Dates Objects into Dates for open_trades_df, ignoring errors
main_open_trades_df['Open Time'] = main_open_trades_df['Open Time'].str.replace('.', '-')
main_open_trades_df['Open Time'] = pd.to_datetime(main_open_trades_df['Open Time'], errors='coerce')

# Sort dataframes
main_closed_trades_df = main_closed_trades_df.sort_values(by='Open Time', ascending=True).reset_index(drop=True)
main_open_trades_df = main_open_trades_df.sort_values(by='Open Time', ascending=True).reset_index(drop=True)

# Filtrar filas con fechas no válidas si lo necesitas
main_closed_trades_df = main_closed_trades_df.dropna(subset=['Open Time', 'Close Time'])
main_open_trades_df = main_open_trades_df.dropna(subset=['Open Time'])


In [4]:
main_closed_trades_df

2,Ticket,Open Time,Type,Size,Item,Price,S / L,T / P,Close Time,Price,Commission,Taxes,Swap,Profit
0,287330602,2023-07-11 00:45:00,buy,0.06,eurusdp,1.10002,0.00000,1.10087,2023-07-11 04:02:39,1.10086,-0.36,0.00,0.00,5.04
1,287331679,2023-07-11 03:15:00,sell,0.06,eurgbpp,0.85547,0.00000,0.85472,2023-07-11 09:03:10,0.85471,-0.46,0.00,0.00,5.88
2,287331770,2023-07-11 03:20:01,sell,0.05,eurjpyp,155.521,0.000,155.446,2023-07-11 03:26:43,155.442,-0.21,0.00,0.00,2.80
3,287331861,2023-07-11 03:25:01,sell,0.05,chfjpyp,159.618,0.000,159.533,2023-07-11 04:21:36,159.533,-0.21,0.00,0.00,3.01
4,287331859,2023-07-11 03:25:01,sell,0.05,cadjpyp,106.431,0.000,106.311,2023-07-11 05:00:54,106.309,-0.21,0.00,0.00,4.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7470,296517210,2024-07-24 16:50:01,buy,0.50,eurjpyp,166.626,0.000,167.343,2024-07-25 03:30:35,166.164,-1.96,0.00,10.20,-150.65
7471,296517605,2024-07-24 16:55:01,buy,6.59,audjpyp,101.297,0.000,101.750,2024-07-24 18:46:56,101.285,-25.76,0.00,0.00,-51.54
7472,296517673,2024-07-24 16:55:02,buy,0.57,eurjpyp,166.637,0.000,167.343,2024-07-25 03:30:31,166.171,-2.23,0.00,11.62,-173.23
7473,296518340,2024-07-24 17:05:00,buy,7.91,audjpyp,101.160,0.000,101.750,2024-07-24 18:46:52,101.285,-30.96,0.00,0.00,644.45


In [5]:
main_open_trades_df

2,Ticket,Open Time,Type,Size,Item,Price,S / L,T / P,Close Time,Price,Commission,Taxes,Swap,Profit
0,295909554,2024-07-01 04:45:00,sell,0.06,eurcadp,1.46945,0.00000,1.48780,NaN,1.50259,-0.26,0.00,0.97,-143.70
1,296063946,2024-07-08 00:41:00,buy,0.04,nzdcadp,0.83875,0.00000,0.82370,NaN,0.81418,-0.18,0.00,0.16,-71.04
2,296125890,2024-07-10 05:05:01,buy,0.05,nzdcadp,0.83157,0.00000,0.82370,NaN,0.81418,-0.22,0.00,0.17,-62.86
3,296127026,2024-07-10 05:25:01,buy,0.05,nzdcadp,0.82869,0.00000,0.82370,NaN,0.81418,-0.22,0.00,0.17,-52.45
4,296164479,2024-07-11 13:15:00,sell,0.04,eurcadp,1.47928,0.00000,1.48780,NaN,1.50259,-0.18,0.00,0.46,-67.38
5,296210435,2024-07-12 13:40:00,sell,0.05,eurcadp,1.48300,0.00000,1.48780,NaN,1.50259,-0.22,0.00,0.59,-70.78
6,296226945,2024-07-12 21:00:00,sell,0.05,eurcadp,1.48632,0.00000,1.48780,NaN,1.50259,-0.22,0.00,0.59,-58.79
7,296228627,2024-07-12 22:50:01,sell,0.04,eurnzdp,1.78254,0.00000,1.82039,NaN,1.84436,-0.15,0.00,1.22,-145.59
8,296246880,2024-07-15 13:00:00,sell,0.06,eurcadp,1.48900,0.00000,1.48780,NaN,1.50259,-0.26,0.00,0.67,-58.92
9,296257027,2024-07-15 17:45:00,sell,0.06,eurcadp,1.49117,0.00000,1.48780,NaN,1.50259,-0.26,0.00,0.67,-49.52


In [6]:
def obtener_archivos_htm(ruta_carpeta): 
    """Obtener una lista de archivos .htm o .html en la carpeta dada."""
    try:
        return [
            os.path.join(ruta_carpeta, archivo)
            for archivo in os.listdir(ruta_carpeta)
            if archivo.endswith(('.htm', '.html')) and os.path.isfile(os.path.join(ruta_carpeta, archivo))
        ]
    except Exception as e:
        print(f"Error al listar archivos en {ruta_carpeta}: {e}")
        return []

def procesar_archivos_secundarios(ruta_carpeta, tamano_bloque=5):
    """Procesar todos los archivos secundarios .htm en la carpeta dada por bloques."""
    archivos = obtener_archivos_htm(ruta_carpeta)
    dataframes = []

    # Procesar archivos en bloques
    for i in range(0, len(archivos), tamano_bloque):
        bloque_archivos = archivos[i:i + tamano_bloque]
        print(f"Procesando bloque de archivos: {bloque_archivos}")

        for archivo in bloque_archivos:
            df = pd.read_html(archivo, header=2)[0]  # Leer con cabeceras
            
            if not df.empty:
                # Renombrar columnas
                df.rename(columns={'Volume': 'Size', 'Price.1': 'Price_1'}, inplace=True)

                # Crear la nueva columna ticket_reference
                df['ticket_reference'] = None

                # Asignar ticket_reference
                for index in range(len(df) - 1):  # Evitar el último índice
                    if isinstance(df.iloc[index + 1]['Profit'], str):
                        match = re.search(r'#(\d+)/', df.iloc[index + 1]['Profit'])
                        if match:
                            df.at[index, 'ticket_reference'] = match.group(1)

                # Eliminar filas donde no hay datos válidos
                df = df.dropna(subset=['ticket_reference'])  # Eliminar filas donde ticket_reference es NaN

                # Agregar la columna 'Archivo' con el nombre limpio del archivo
                nombre_archivo = os.path.basename(archivo)  # Obtener solo el nombre del archivo
                nombre_archivo_limpio = nombre_archivo.replace('Statement_', '').replace('StatementDetailed_', '').replace('.htm', '')
                df['Archivo'] = nombre_archivo_limpio  # Asignar el nombre limpio a la nueva columna

                dataframes.append(df)
            else:
                print(f"El archivo {archivo} está vacío o no contiene datos válidos.")

    # Concatenar todos los DataFrames procesados en uno solo
    if dataframes:
        df_combinado = pd.concat(dataframes, ignore_index=True)
        return df_combinado
    else:
        return pd.DataFrame()  # Retornar un DataFrame vacío si no hay datos

# Ejemplo de uso
if __name__ == "__main__":
    ruta_carpeta = '/work/notebook/Files'  # Cambia esto a la ruta de tu carpeta
    tamano_bloque = 5  # Cambia el tamaño del bloque según sea necesario
    df_secundarios = procesar_archivos_secundarios(ruta_carpeta, tamano_bloque)



Procesando bloque de archivos: ['/work/notebook/Files/StatementDetailed_2100083468.htm', '/work/notebook/Files/StatementDetailed_2100103082.htm', '/work/notebook/Files/StatementDetailed_2100104544.htm', '/work/notebook/Files/StatementDetailed_2100106463.htm', '/work/notebook/Files/StatementDetailed_2100110572.htm']
Procesando bloque de archivos: ['/work/notebook/Files/StatementDetailed_2100110711.htm', '/work/notebook/Files/StatementDetailed_2100110876.htm', '/work/notebook/Files/StatementDetailed_2100112045.htm', '/work/notebook/Files/StatementDetailed_2100112094 (2).htm', '/work/notebook/Files/StatementDetailed_2100112540 (2).htm']
Procesando bloque de archivos: ['/work/notebook/Files/StatementDetailed_2100112553 (3).htm', '/work/notebook/Files/StatementDetailed_2100113110.htm', '/work/notebook/Files/StatementDetailed_2100113365.htm', '/work/notebook/Files/StatementDetailed_2100113372.htm', '/work/notebook/Files/StatementDetailed_2100113558.htm']
Procesando bloque de archivos: ['/wor

In [7]:
df_secundarios

,Ticket,Open Time,Type,Size,Item,Price,S / L,T / P,Close Time,Price_1,Commission,Taxes,Swap,Profit,ticket_reference,Archivo
0,295168983,2024.05.31 10:58:04,buy,0.06,nzdusdp,0.61192,0.00000,0.00000,2024.07.31 20:17:41,0.59441,-0.36,0.00,-6.29,-105.06,295168722,2100103082
1,295532687,2024.06.13 20:35:01,buy,0.07,eurcadp,1.47672,0.00000,0.00000,2024.07.01 03:21:10,1.46915,-0.31,0.00,-5.70,-38.75,295532671,2100103082
2,295539634,2024.06.14 06:45:01,buy,0.06,nzdcadp,0.84553,0.00000,0.00000,2024.07.05 20:47:40,0.83732,-0.26,0.00,0.34,-36.12,295539616,2100103082
3,295615988,2024.06.18 10:45:01,buy,0.07,nzdcadp,0.83906,0.00000,0.00000,2024.07.05 20:47:44,0.83733,-0.31,0.00,0.33,-8.87,295615972,2100103082
4,295651807,2024.06.19 12:30:01,sell,0.06,cadjpyp,115.071,0.000,0.000,2024.07.08 02:49:24,117.740,-0.23,0.00,-16.55,-99.72,295651711,2100103082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73094,296339720,2024.07.17 19:47:11,buy,0.02,cadchfp,0.64570,0.00000,0.00000,2024.07.31 10:04:31,0.63703,-0.14,0.00,0.67,-19.67,296339669,2100111968
73095,296497423,2024.07.24 07:35:00,buy,0.02,audchfp,0.58707,0.00000,0.00000,2024.07.31 10:04:31,0.57313,-0.13,0.00,0.42,-31.62,296497407,2100111968
73096,296319317,2024.07.17 11:35:01,sell,0.03,eurcadp,1.49326,0.00000,0.00000,2024.07.31 10:04:32,1.49694,-0.13,0.00,0.34,-7.97,296319305,2100111968
73097,295168982,2024.05.31 10:58:03,buy,0.01,nzdusdp,0.61192,0.00000,0.00000,2024.07.31 20:17:30,0.59440,-0.06,0.00,-1.14,-17.52,295168722,2100111968


In [8]:
# Crear copias de los DataFrames existentes
closed_trades_copy = main_closed_trades_df.copy()
open_trades_copy = main_open_trades_df.copy()

# Combinar los DataFrames copiados en un nuevo DataFrame
df_combined = pd.concat([closed_trades_copy, open_trades_copy], ignore_index=True)

# Convertir la columna Price a float (asegúrate de que los valores sean convertibles)
df_combined['Price'] = df_combined['Price'].astype(float)

# Crear la columna 'ticket_reference' en df_combined y establecerla como NaN
df_combined['ticket_reference'] = pd.NA

# Crear la columna 'Archivo' y asignarle el valor "Cuenta_Madre"
df_combined['Archivo'] = "Cuenta_Madre"

# Renombrar las columnas 'Price' para evitar duplicados
# Asegurarte de que la lista de nuevos nombres tenga la misma longitud
new_column_names = [
    'Ticket', 'Open Time', 'Type', 'Size', 'Item', 
    'Price', 'S / L', 'T / P', 'Close Time', 
    'Price_1', 'Commission', 'Taxes', 'Swap', 'Profit', 'ticket_reference', 'Archivo'
]

# Comprobar si la longitud coincide
if len(new_column_names) == len(df_combined.columns):
    df_combined.columns = new_column_names
else:
    print("Error: La longitud de los nuevos nombres no coincide con el número de columnas en df_combined.")



/tmp/ipykernel_84/4282168537.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_combined = pd.concat([closed_trades_copy, open_trades_copy], ignore_index=True)


In [9]:
df_combined

,Ticket,Open Time,Type,Size,Item,Price,S / L,T / P,Close Time,Price_1,Commission,Taxes,Swap,Profit,ticket_reference,Archivo
0,287330602,2023-07-11 00:45:00,buy,0.06,eurusdp,1.10002,0.00000,1.10087,2023-07-11 04:02:39,1.10086,-0.36,0.00,0.00,5.04,NaN,Cuenta_Madre
1,287331679,2023-07-11 03:15:00,sell,0.06,eurgbpp,0.85547,0.00000,0.85472,2023-07-11 09:03:10,0.85471,-0.46,0.00,0.00,5.88,NaN,Cuenta_Madre
2,287331770,2023-07-11 03:20:01,sell,0.05,eurjpyp,155.52100,0.000,155.446,2023-07-11 03:26:43,155.44200,-0.21,0.00,0.00,2.80,NaN,Cuenta_Madre
3,287331861,2023-07-11 03:25:01,sell,0.05,chfjpyp,159.61800,0.000,159.533,2023-07-11 04:21:36,159.53300,-0.21,0.00,0.00,3.01,NaN,Cuenta_Madre
4,287331859,2023-07-11 03:25:01,sell,0.05,cadjpyp,106.43100,0.000,106.311,2023-07-11 05:00:54,106.30900,-0.21,0.00,0.00,4.33,NaN,Cuenta_Madre
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7517,296493992,2024-07-24 06:25:01,buy,0.16,nzdcadp,0.81753,0.00000,0.82370,NaT,0.81418,-0.70,0.00,0.10,-38.74,NaN,Cuenta_Madre
7518,296497137,2024-07-24 07:30:00,buy,0.19,nzdcadp,0.81679,0.00000,0.82370,NaT,0.81418,-0.83,0.00,0.11,-35.85,NaN,Cuenta_Madre
7519,296497407,2024-07-24 07:35:00,buy,0.04,audchfp,0.58706,0.00000,0.59097,NaT,0.57850,-0.27,0.00,0.60,-38.75,NaN,Cuenta_Madre
7520,296497685,2024-07-24 07:40:01,buy,0.06,nzdusdp,0.59172,0.00000,0.59535,NaT,0.58858,-0.36,0.00,-0.50,-18.84,NaN,Cuenta_Madre


In [10]:
# Crear copias de df_combined y df_secundarios
combined_copy = df_combined.copy()
secundarios_copy = df_secundarios.copy()

# Inicializar una lista para almacenar las filas intercaladas
intercalado = []

# Iterar sobre las filas de combined_copy
for index, row in combined_copy.iterrows():
    # Agregar la fila de df_combined
    intercalado.append(row)

    # Buscar coincidencias en secundarios_copy
    coincidencias_secundarios = secundarios_copy[secundarios_copy['ticket_reference'] == row['Ticket']]
    
    # Agregar todas las coincidencias a la lista
    for _, coincidencia in coincidencias_secundarios.iterrows():
        intercalado.append(coincidencia)  # Agregar cada coincidencia

# Convertir la lista intercalada en un DataFrame
df_final_intercalado = pd.DataFrame(intercalado)

# Reiniciar el índice del DataFrame final
df_final_intercalado.reset_index(drop=True, inplace=True)

# Filtrar para mantener solo los registros con coincidencias
df_final_intercalado = df_final_intercalado[
    df_final_intercalado['Ticket'].isin(secundarios_copy['ticket_reference']) |
    df_final_intercalado['ticket_reference'].isin(combined_copy['Ticket'])
]

# Crear un DataFrame para los registros que no tienen coincidencias
no_coincidencias = combined_copy[~combined_copy['Ticket'].isin(secundarios_copy['ticket_reference'])]



In [11]:
df_final_intercalado

,Ticket,Open Time,Type,Size,Item,Price,S / L,T / P,Close Time,Price_1,Commission,Taxes,Swap,Profit,ticket_reference,Archivo
680,287966548,2023-08-07 08:30:01,buy,0.03,audchfp,0.5749,0.00000,0.57454,2023-08-10 15:30:06,0.5744,-0.21,0.00,0.25,-1.72,<NA>,Cuenta_Madre
681,288056369,2023.08.10 00:44:41,buy,0.01,audchfp,0.57302,0.00000,0.00000,2023.08.10 15:30:06,0.57446,-0.07,0.00,0.00,1.65,287966548,2100107024
711,287996492,2023-08-08 08:55:03,buy,0.03,audchfp,0.57068,0.00000,0.57454,2023-08-10 15:30:06,0.57444,-0.21,0.00,0.20,12.94,<NA>,Cuenta_Madre
712,288056368,2023.08.10 00:44:41,buy,0.01,audchfp,0.57302,0.00000,0.00000,2023.08.10 15:30:07,0.57446,-0.07,0.00,0.00,1.65,287996492,2100107024
719,288007092,2023-08-08 14:25:00,sell,0.05,audcadp,0.87547,0.00000,0.87705,2023-08-10 20:36:43,0.87705,-0.22,0.00,0.03,-5.88,<NA>,Cuenta_Madre
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80616,296498618,2024.07.24 07:50:01,buy,0.04,eurchfp,0.96672,0.00000,0.00000,2024.07.31 10:04:21,0.95375,-0.27,0.00,1.16,-58.84,296498584,2100110875
80617,296498587,2024.07.24 07:50:01,buy,0.01,eurchfp,0.96672,0.00000,0.00000,2024.07.31 10:04:21,0.95375,-0.07,0.00,0.28,-14.71,296498584,2100110879
80618,296498589,2024.07.24 07:50:01,buy,0.06,eurchfp,0.96672,0.00000,0.00000,2024.07.31 10:04:21,0.95375,-0.40,0.00,1.74,-88.26,296498584,2100111378
80619,296498592,2024.07.24 07:50:01,buy,0.01,eurchfp,0.96672,0.00000,0.00000,2024.07.31 10:04:21,0.95375,-0.07,0.00,0.28,-14.71,296498584,2100111724


In [12]:
no_coincidencias

,Ticket,Open Time,Type,Size,Item,Price,S / L,T / P,Close Time,Price_1,Commission,Taxes,Swap,Profit,ticket_reference,Archivo
0,287330602,2023-07-11 00:45:00,buy,0.06,eurusdp,1.10002,0.00000,1.10087,2023-07-11 04:02:39,1.10086,-0.36,0.00,0.00,5.04,<NA>,Cuenta_Madre
1,287331679,2023-07-11 03:15:00,sell,0.06,eurgbpp,0.85547,0.00000,0.85472,2023-07-11 09:03:10,0.85471,-0.46,0.00,0.00,5.88,<NA>,Cuenta_Madre
2,287331770,2023-07-11 03:20:01,sell,0.05,eurjpyp,155.52100,0.000,155.446,2023-07-11 03:26:43,155.44200,-0.21,0.00,0.00,2.80,<NA>,Cuenta_Madre
3,287331861,2023-07-11 03:25:01,sell,0.05,chfjpyp,159.61800,0.000,159.533,2023-07-11 04:21:36,159.53300,-0.21,0.00,0.00,3.01,<NA>,Cuenta_Madre
4,287331859,2023-07-11 03:25:01,sell,0.05,cadjpyp,106.43100,0.000,106.311,2023-07-11 05:00:54,106.30900,-0.21,0.00,0.00,4.33,<NA>,Cuenta_Madre
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5545,293628119,2024-04-08 06:05:00,sell,0.04,chfjpyp,168.07700,0.000,167.992,2024-04-08 06:35:56,167.98900,-0.16,0.00,0.00,2.32,<NA>,Cuenta_Madre
5546,293628190,2024-04-08 06:10:00,sell,0.04,audjpyp,99.91900,0.000,99.819,2024-04-08 06:20:24,99.82000,-0.16,0.00,0.00,2.61,<NA>,Cuenta_Madre
5547,293628752,2024-04-08 06:45:00,buy,0.05,audusdp,0.65735,0.00000,0.65840,2024-04-08 10:15:23,0.65840,-0.30,0.00,0.00,5.25,<NA>,Cuenta_Madre
5548,293629037,2024-04-08 07:05:00,buy,0.05,eurusdp,1.08351,0.00000,1.08436,2024-04-08 16:33:33,1.08437,-0.30,0.00,0.00,4.30,<NA>,Cuenta_Madre


In [13]:
# Crear una copia de df_final_intercalado 
df_final_intercalado_copy = df_final_intercalado.copy()

# Filtrar los registros de "Cuenta_Madre"
cuenta_madre = df_final_intercalado_copy[df_final_intercalado_copy['Archivo'] == 'Cuenta_Madre']

# Inicializar una lista para almacenar los nuevos registros
nuevos_registros = []

# Iterar sobre las filas de cuenta_madre
for index, row in cuenta_madre.iterrows():
    # Buscar las filas correspondientes en ticket_reference
    coincidencias = df_final_intercalado_copy[df_final_intercalado_copy['ticket_reference'] == row['Ticket']]
    
    # Agregar todas las coincidencias, precediendo cada coincidencia con la fila de Cuenta_Madre
    for _, coincidencia in coincidencias.iterrows():
        nuevos_registros.append(row)       # Agregar la fila de Cuenta_Madre
        nuevos_registros.append(coincidencia)  # Agregar la coincidencia de ticket_reference

# Convertir la lista de nuevos registros en un DataFrame
df_final_duplicado = pd.DataFrame(nuevos_registros)

# Reiniciar el índice del DataFrame final
df_final_duplicado.reset_index(drop=True, inplace=True)



In [14]:

df_final_duplicado

,Ticket,Open Time,Type,Size,Item,Price,S / L,T / P,Close Time,Price_1,Commission,Taxes,Swap,Profit,ticket_reference,Archivo
0,287966548,2023-08-07 08:30:01,buy,0.03,audchfp,0.5749,0.00000,0.57454,2023-08-10 15:30:06,0.5744,-0.21,0.00,0.25,-1.72,<NA>,Cuenta_Madre
1,288056369,2023.08.10 00:44:41,buy,0.01,audchfp,0.57302,0.00000,0.00000,2023.08.10 15:30:06,0.57446,-0.07,0.00,0.00,1.65,287966548,2100107024
2,287996492,2023-08-08 08:55:03,buy,0.03,audchfp,0.57068,0.00000,0.57454,2023-08-10 15:30:06,0.57444,-0.21,0.00,0.20,12.94,<NA>,Cuenta_Madre
3,288056368,2023.08.10 00:44:41,buy,0.01,audchfp,0.57302,0.00000,0.00000,2023.08.10 15:30:07,0.57446,-0.07,0.00,0.00,1.65,287996492,2100107024
4,288007092,2023-08-08 14:25:00,sell,0.05,audcadp,0.87547,0.00000,0.87705,2023-08-10 20:36:43,0.87705,-0.22,0.00,0.03,-5.88,<NA>,Cuenta_Madre
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146193,296498589,2024.07.24 07:50:01,buy,0.06,eurchfp,0.96672,0.00000,0.00000,2024.07.31 10:04:21,0.95375,-0.40,0.00,1.74,-88.26,296498584,2100111378
146194,296498584,2024-07-24 07:50:01,buy,0.04,eurchfp,0.96672,0.00000,0.96747,NaT,0.95853,-0.27,0.00,0.84,-37.08,<NA>,Cuenta_Madre
146195,296498592,2024.07.24 07:50:01,buy,0.01,eurchfp,0.96672,0.00000,0.00000,2024.07.31 10:04:21,0.95375,-0.07,0.00,0.28,-14.71,296498584,2100111724
146196,296498584,2024-07-24 07:50:01,buy,0.04,eurchfp,0.96672,0.00000,0.96747,NaT,0.95853,-0.27,0.00,0.84,-37.08,<NA>,Cuenta_Madre


In [15]:
# Crear una copia del dataframe df_final_duplicado para evitar modificarlo directamente
df_final_duplicado_copy = df_final_duplicado.copy()

# Inicializar una lista para almacenar los nuevos registros combinados
nuevos_registros_combinados = []

# Iterar sobre las filas del DataFrame
for index in range(0, len(df_final_duplicado_copy), 2):  # Iterar de dos en dos
    if index + 1 < len(df_final_duplicado_copy):  # Asegurarse de que hay una fila siguiente
        # Obtener la fila actual y la siguiente
        fila_ticket = df_final_duplicado_copy.iloc[index]
        fila_reference = df_final_duplicado_copy.iloc[index + 1]

        # Combinar las filas
        fila_combinada = fila_ticket.copy()  # Copia de la fila de "Ticket"
        
        # Añadir columnas de la fila secundaria con el sufijo '_Secundario'
        for col in fila_reference.index:
            fila_combinada[f"{col}_Secundario"] = fila_reference[col]  # Añadir la columna secundaria con el sufijo

        # Agregar la fila combinada a la lista
        nuevos_registros_combinados.append(fila_combinada)

# Convertir la lista de nuevos registros combinados en un DataFrame
df_final_combined = pd.DataFrame(nuevos_registros_combinados)

# Reiniciar el índice del DataFrame final
df_final_combined.reset_index(drop=True, inplace=True)

# Omitir la columna 'ticket_reference' al mostrar el DataFrame
df_final_combined_visualizado = df_final_combined.drop(columns=['ticket_reference'], errors='ignore')



In [16]:
df_final_combined_visualizado

,Ticket,Open Time,Type,Size,Item,Price,S / L,T / P,Close Time,Price_1,...,S / L_Secundario,T / P_Secundario,Close Time_Secundario,Price_1_Secundario,Commission_Secundario,Taxes_Secundario,Swap_Secundario,Profit_Secundario,ticket_reference_Secundario,Archivo_Secundario
0,287966548,2023-08-07 08:30:01,buy,0.03,audchfp,0.57490,0.00000,0.57454,2023-08-10 15:30:06,0.57440,...,0.00000,0.00000,2023.08.10 15:30:06,0.57446,-0.07,0.00,0.00,1.65,287966548,2100107024
1,287996492,2023-08-08 08:55:03,buy,0.03,audchfp,0.57068,0.00000,0.57454,2023-08-10 15:30:06,0.57444,...,0.00000,0.00000,2023.08.10 15:30:07,0.57446,-0.07,0.00,0.00,1.65,287996492,2100107024
2,288007092,2023-08-08 14:25:00,sell,0.05,audcadp,0.87547,0.00000,0.87705,2023-08-10 20:36:43,0.87705,...,0.00000,0.00000,2023.08.10 20:36:43,0.87705,-0.04,0.00,0.00,-0.73,288007092,2100107024
3,288017886,2023-08-08 18:20:01,sell,0.03,eurjpyp,156.84000,0.000,158.363,2023-08-14 04:03:07,158.35500,...,0.000,0.000,2023.08.14 04:03:07,158.355,-0.04,0.00,-0.24,-4.64,288017886,2100107024
4,288019124,2023-08-08 19:25:00,sell,0.03,cadjpyp,106.55000,0.000,107.586,2023-08-17 22:17:49,107.58600,...,0.000,0.000,2023.08.17 22:17:50,107.587,-0.04,0.00,-0.30,-3.65,288019124,2100107024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73094,296498584,2024-07-24 07:50:01,buy,0.04,eurchfp,0.96672,0.00000,0.96747,NaT,0.95853,...,0.00000,0.00000,2024.07.31 10:04:21,0.95375,-0.27,0.00,1.16,-58.84,296498584,2100110875
73095,296498584,2024-07-24 07:50:01,buy,0.04,eurchfp,0.96672,0.00000,0.96747,NaT,0.95853,...,0.00000,0.00000,2024.07.31 10:04:21,0.95375,-0.07,0.00,0.28,-14.71,296498584,2100110879
73096,296498584,2024-07-24 07:50:01,buy,0.04,eurchfp,0.96672,0.00000,0.96747,NaT,0.95853,...,0.00000,0.00000,2024.07.31 10:04:21,0.95375,-0.40,0.00,1.74,-88.26,296498584,2100111378
73097,296498584,2024-07-24 07:50:01,buy,0.04,eurchfp,0.96672,0.00000,0.96747,NaT,0.95853,...,0.00000,0.00000,2024.07.31 10:04:21,0.95375,-0.07,0.00,0.28,-14.71,296498584,2100111724


In [17]:
# Crear una copia del dataframe df_final_combined para evitar modificarlo directamente
df_final_combined_copy = df_final_combined_visualizado.copy()

# Asegúrate de que las columnas 'Open Time' y 'Close Time' estén en formato datetime
df_final_combined_copy['Open Time'] = pd.to_datetime(df_final_combined_copy['Open Time'], errors='coerce')
df_final_combined_copy['Close Time'] = pd.to_datetime(df_final_combined_copy['Close Time'], errors='coerce')
df_final_combined_copy['Open Time_Secundario'] = pd.to_datetime(df_final_combined_copy['Open Time_Secundario'], errors='coerce')
df_final_combined_copy['Close Time_Secundario'] = pd.to_datetime(df_final_combined_copy['Close Time_Secundario'], errors='coerce')

# Inicializar las nuevas columnas de desfase
df_final_combined_copy['Desfase_Open'] = pd.NA
df_final_combined_copy['Desfase_Close'] = pd.NA

# Calcular los desfases entre Open Time y Open Time_Secundario, Close Time y Close Time_Secundario
for index in range(len(df_final_combined_copy)):
    # Calcular el desfase entre Open Time y Open Time_Secundario
    if pd.notna(df_final_combined_copy['Open Time'][index]) and pd.notna(df_final_combined_copy['Open Time_Secundario'][index]):
        desfase_open = (df_final_combined_copy['Open Time_Secundario'][index] - df_final_combined_copy['Open Time'][index]).total_seconds()
        df_final_combined_copy.at[index, 'Desfase_Open'] = desfase_open

    # Calcular el desfase entre Close Time y Close Time_Secundario
    if pd.notna(df_final_combined_copy['Close Time'][index]) and pd.notna(df_final_combined_copy['Close Time_Secundario'][index]):
        desfase_close = (df_final_combined_copy['Close Time_Secundario'][index] - df_final_combined_copy['Close Time'][index]).total_seconds()
        df_final_combined_copy.at[index, 'Desfase_Close'] = desfase_close


In [18]:
df_final_combined_copy

,Ticket,Open Time,Type,Size,Item,Price,S / L,T / P,Close Time,Price_1,...,Close Time_Secundario,Price_1_Secundario,Commission_Secundario,Taxes_Secundario,Swap_Secundario,Profit_Secundario,ticket_reference_Secundario,Archivo_Secundario,Desfase_Open,Desfase_Close
0,287966548,2023-08-07 08:30:01,buy,0.03,audchfp,0.57490,0.00000,0.57454,2023-08-10 15:30:06,0.57440,...,2023-08-10 15:30:06,0.57446,-0.07,0.00,0.00,1.65,287966548,2100107024,231280.0,0.0
1,287996492,2023-08-08 08:55:03,buy,0.03,audchfp,0.57068,0.00000,0.57454,2023-08-10 15:30:06,0.57444,...,2023-08-10 15:30:07,0.57446,-0.07,0.00,0.00,1.65,287996492,2100107024,143378.0,1.0
2,288007092,2023-08-08 14:25:00,sell,0.05,audcadp,0.87547,0.00000,0.87705,2023-08-10 20:36:43,0.87705,...,2023-08-10 20:36:43,0.87705,-0.04,0.00,0.00,-0.73,288007092,2100107024,123581.0,0.0
3,288017886,2023-08-08 18:20:01,sell,0.03,eurjpyp,156.84000,0.000,158.363,2023-08-14 04:03:07,158.35500,...,2023-08-14 04:03:07,158.355,-0.04,0.00,-0.24,-4.64,288017886,2100107024,109480.0,0.0
4,288019124,2023-08-08 19:25:00,sell,0.03,cadjpyp,106.55000,0.000,107.586,2023-08-17 22:17:49,107.58600,...,2023-08-17 22:17:50,107.587,-0.04,0.00,-0.30,-3.65,288019124,2100107024,105581.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73094,296498584,2024-07-24 07:50:01,buy,0.04,eurchfp,0.96672,0.00000,0.96747,NaT,0.95853,...,2024-07-31 10:04:21,0.95375,-0.27,0.00,1.16,-58.84,296498584,2100110875,0.0,<NA>
73095,296498584,2024-07-24 07:50:01,buy,0.04,eurchfp,0.96672,0.00000,0.96747,NaT,0.95853,...,2024-07-31 10:04:21,0.95375,-0.07,0.00,0.28,-14.71,296498584,2100110879,0.0,<NA>
73096,296498584,2024-07-24 07:50:01,buy,0.04,eurchfp,0.96672,0.00000,0.96747,NaT,0.95853,...,2024-07-31 10:04:21,0.95375,-0.40,0.00,1.74,-88.26,296498584,2100111378,0.0,<NA>
73097,296498584,2024-07-24 07:50:01,buy,0.04,eurchfp,0.96672,0.00000,0.96747,NaT,0.95853,...,2024-07-31 10:04:21,0.95375,-0.07,0.00,0.28,-14.71,296498584,2100111724,0.0,<NA>


In [19]:
# Crear una copia del dataframe df_final_combined_copy para evitar modificarlo directamente
df_final_combined_copy_copy = df_final_combined_copy.copy()

# Inicializar una lista para almacenar los registros filtrados
registros_filtrados = []

# Filtrar los registros que tengan más de 2 segundos en Desfase_Open o Desfase_Close
for index in range(len(df_final_combined_copy_copy)):
    if (pd.notna(df_final_combined_copy_copy['Desfase_Open'][index]) and df_final_combined_copy_copy['Desfase_Open'][index] > 2) or \
       (pd.notna(df_final_combined_copy_copy['Desfase_Close'][index]) and df_final_combined_copy_copy['Desfase_Close'][index] > 2):
        registros_filtrados.append(df_final_combined_copy_copy.iloc[index])  # Agregar el registro que cumple la condición

# Convertir la lista de registros filtrados en un DataFrame
df_filtrado = pd.DataFrame(registros_filtrados)

# Reiniciar el índice del DataFrame filtrado
df_filtrado.reset_index(drop=True, inplace=True)


In [20]:
df_filtrado

,Ticket,Open Time,Type,Size,Item,Price,S / L,T / P,Close Time,Price_1,...,Close Time_Secundario,Price_1_Secundario,Commission_Secundario,Taxes_Secundario,Swap_Secundario,Profit_Secundario,ticket_reference_Secundario,Archivo_Secundario,Desfase_Open,Desfase_Close
0,287966548,2023-08-07 08:30:01,buy,0.03,audchfp,0.57490,0.00000,0.57454,2023-08-10 15:30:06,0.57440,...,2023-08-10 15:30:06,0.57446,-0.07,0.00,0.00,1.65,287966548,2100107024,231280.0,0.0
1,287996492,2023-08-08 08:55:03,buy,0.03,audchfp,0.57068,0.00000,0.57454,2023-08-10 15:30:06,0.57444,...,2023-08-10 15:30:07,0.57446,-0.07,0.00,0.00,1.65,287996492,2100107024,143378.0,1.0
2,288007092,2023-08-08 14:25:00,sell,0.05,audcadp,0.87547,0.00000,0.87705,2023-08-10 20:36:43,0.87705,...,2023-08-10 20:36:43,0.87705,-0.04,0.00,0.00,-0.73,288007092,2100107024,123581.0,0.0
3,288017886,2023-08-08 18:20:01,sell,0.03,eurjpyp,156.84000,0.000,158.363,2023-08-14 04:03:07,158.35500,...,2023-08-14 04:03:07,158.355,-0.04,0.00,-0.24,-4.64,288017886,2100107024,109480.0,0.0
4,288019124,2023-08-08 19:25:00,sell,0.03,cadjpyp,106.55000,0.000,107.586,2023-08-17 22:17:49,107.58600,...,2023-08-17 22:17:50,107.587,-0.04,0.00,-0.30,-3.65,288019124,2100107024,105581.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10578,296494007,2024-07-24 06:25:01,buy,0.06,nzdusdp,0.59265,0.00000,0.59535,NaT,0.58858,...,2024-07-31 10:04:04,0.59186,-0.48,0.00,-0.94,-6.48,296494007,2100111378,3.0,NaN
10579,296494007,2024-07-24 06:25:01,buy,0.06,nzdusdp,0.59265,0.00000,0.59535,NaT,0.58858,...,2024-07-31 10:04:04,0.59186,-0.12,0.00,-0.24,-1.62,296494007,2100111724,3.0,NaN
10580,296493992,2024-07-24 06:25:01,buy,0.16,nzdcadp,0.81753,0.00000,0.82370,NaT,0.81418,...,2024-07-31 10:04:07,0.81919,-0.17,0.00,0.04,4.65,296493992,2100125435,3.0,NaN
10581,296493992,2024-07-24 06:25:01,buy,0.16,nzdcadp,0.81753,0.00000,0.82370,NaT,0.81418,...,2024-07-31 10:04:07,0.81919,-0.22,0.00,0.06,5.81,296493992,2100125501,3.0,NaN


In [21]:
# Crear una copia del dataframe df_final para evitar modificarlo directamente 
df_final_copy = df_filtrado.copy()

# Crear la carpeta para guardar los archivos Excel
carpeta_reportes = 'reportes_por_archivo'
os.makedirs(carpeta_reportes, exist_ok=True)  # Crear carpeta si no existe

# Generar un archivo Excel para cada Archivo_Secundario
for archivo, grupo in df_final_copy.groupby('Archivo_Secundario'):
    # Nombre del archivo basado en el archivo
    nombre_archivo = os.path.join(carpeta_reportes, f'reporte_{archivo}.xlsx')
    
    # Guardar el grupo completo en un archivo Excel
    grupo.to_excel(nombre_archivo, index=False)

# Imprimir mensaje de finalización
print("Reportes generados y guardados en la carpeta:", carpeta_reportes)


Reportes generados y guardados en la carpeta: reportes_por_archivo


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7c4c5df4-a923-4ae8-9bc1-a7df7bd3ef3b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>